In [ ]:
!nvcc --version 
!pip install git+https://github.com/afnan47/cuda.git 
%load_ext nvcc_plugin

In [ ]:
%%cu

In [ ]:
#include <iostream> #include <cuda.h> using namespace std; #define BLOCK_SIZE 2 __global__ void gpuMM(float *A, float *B, float *C, int N) { // Matrix multiplication for NxN matrices C=A*B // Each thread computes a single element of C int row = blockIdx.y*blockDim.y + threadIdx.y; int col = blockIdx.x*blockDim.x + threadIdx.x; float sum = 0.f;
for (int n = 0; n < N; ++n) sum += A[row*N+n]*B[n*N+col]; C[row*N+col] = sum; } int main(int argc, char *argv[]) {int N;float K; // Perform matrix multiplication C = A*B // where A, B and C are NxN matrices // Restricted to matrices where N = K*BLOCK_SIZE; cout<<"Enter a Value for Size/2 of matrix"; cin>>K; K = 1; N = K*BLOCK_SIZE;cout << "\n Executing Matrix Multiplcation" << endl; cout << "\n Matrix size: " << N << "x" << N << endl; // Allocate memory on the host float *hA,*hB,*hC; hA = new float[N*N]; hB = new float[N*N]; hC = new float[N*N]; // Initialize matrices on the host for (int j=0; j<N; j++){ for (int i=0; i<N; i++){ hA[j*N+i] = 2; hB[j*N+i] = 4; } }
// Allocate memory on the device int size = N*N*sizeof(float); // Size of the memory in bytes float *dA,*dB,*dC; cudaMalloc(&dA,size); cudaMalloc(&dB,size); cudaMalloc(&dC,size);dim3 threadBlock(BLOCK_SIZE,BLOCK_SIZE); dim3 grid(K,K); cout<<"\n Input Matrix 1 \n"; for (int row=0; row<N; row++){ for (int col=0; col<N; col++){ cout<<hA[row*col]<<" "; } cout<<endl; } cout<<"\n Input Matrix 2 \n"; for (int row=0; row<N; row++){ for (int col=0; col<N; col++){ cout<<hB[row*col]<<" "; } cout<<endl; } // Copy matrices from the host to device cudaMemcpy(dA,hA,size,cudaMemcpyHostToDevice); cudaMemcpy(dB,hB,size,cudaMemcpyHostToDevice); //Execute the matrix multiplication kernel gpuMM<<<grid,threadBlock>>>(dA,dB,dC,N);// Now do the matrix multiplication on the CPU /*float sum; for (int row=0; row<N; row++){ for (int col=0; col<N; col++){ sum = 0.f; for (int n=0; n<N; n++){ sum += hA[row*N+n]*hB[n*N+col]; } hC[row*N+col] = sum; cout << sum <<" "; } cout<<endl; }*/ // Allocate memory to store the GPU answer on the host float *C; C = new float[N*N]; // Now copy the GPU result back to CPU cudaMemcpy(C,dC,size,cudaMemcpyDeviceToHost); // Check the result and make sure it is correct cout <<"\n\n\n\n\n Resultant matrix\n\n"; for (int row=0; row<N; row++){ for (int col=0; col<N; col++){ cout<<C[row*col]<<" "; } cout<<endl; } cout << "Finished." << endl; }

In [ ]:
import torch
import numpy as np
import time

# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Set dimensions for the matrices
M = 3  # rows of A
N = 4  # columns of A, rows of B
K = 2  # columns of B

# Create random matrices A and B
# A: M x N matrix
# B: N x K matrix
# C: M x K matrix (result)

# Create matrices with random integers between 0 and 9
A = torch.randint(0, 10, (M, N), dtype=torch.float32)
B = torch.randint(0, 10, (N, K), dtype=torch.float32)

print(f"Matrix A ({M}x{N}):")
print(A)
print(f"\nMatrix B ({N}x{K}):")
print(B)

# Move matrices to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
A_gpu = A.to(device)
B_gpu = B.to(device)

# Perform matrix multiplication on GPU
start_time = time.time()
C_gpu = torch.matmul(A_gpu, B_gpu)
torch.cuda.synchronize()  # Wait for GPU computation to finish
gpu_time = time.time() - start_time

# For comparison, also perform matrix multiplication on CPU
start_time = time.time()
C_cpu = torch.matmul(A, B)
cpu_time = time.time() - start_time

# Move result back to CPU for printing
C = C_gpu.cpu()

print(f"\nMatrix Multiplication Result C = A × B ({M}x{K}):")
print(C)

# Verify results match
if torch.allclose(C, C_cpu):
    print("\nGPU and CPU results match!")
else:
    print("\nWarning: GPU and CPU results differ!")

print(f"GPU time: {gpu_time:.6f} seconds")
print(f"CPU time: {cpu_time:.6f} seconds")
print(f"Speedup: {cpu_time/gpu_time:.2f}x")

# For larger matrices where the speedup will be more noticeable
print("\n--- Testing with larger matrices ---")
large_M, large_N, large_K = 1000, 1500, 2000

print(f"Creating large matrices: A({large_M}x{large_N}) and B({large_N}x{large_K})...")
large_A = torch.rand((large_M, large_N), dtype=torch.float32)
large_B = torch.rand((large_N, large_K), dtype=torch.float32)

# CPU multiplication with large matrices
start_time = time.time()
large_C_cpu = torch.matmul(large_A, large_B)
large_cpu_time = time.time() - start_time
print(f"CPU time for large matrices: {large_cpu_time:.6f} seconds")

# GPU multiplication with large matrices
large_A_gpu = large_A.to(device)
large_B_gpu = large_B.to(device)

start_time = time.time()
large_C_gpu = torch.matmul(large_A_gpu, large_B_gpu)
torch.cuda.synchronize()
large_gpu_time = time.time() - start_time
print(f"GPU time for large matrices: {large_gpu_time:.6f} seconds")
print(f"Large matrix speedup: {large_cpu_time/large_gpu_time:.2f}x")

# Output the dimensions of the result to verify correctness
print(f"Result matrix dimensions: {large_C_gpu.size()}")